# 爬蟲練習 - 爬取Dcard文章
[Dcard]((https://www.dcard.tw/f))是非常適合練習爬蟲的網站，<br>
除了Dcard台灣熱門的社群網站之外，Dcard也提供了非常便利的API讓我們能從網站上爬下文章。

這邊文章的架構如下
- 嘗試抓取一篇Dcard的文章<br>
  具體項目如下：編號 / 標題 / 引言 / 內容 / 發布時間 / 更新時間...等等
- 一次爬100篇Dcard文章<br>
  透過系統提供的api，一次抓取100篇熱門文章
- 爬超過100篇Dcard文章<br>
  因為API限制一次最多100篇，在這裡我們透過簡單的迴圈一次爬1000篇文章。


在這裡我們先練習爬文章內容的方法，若想進一步爬文章底下留言的人，可以參考補充資料中的範例，以下我們就開始練習吧！

補充資料：[爬蟲-從dcard網站看爬蟲入門-iii](https://medium.com/pyladies-taiwan/%E7%88%AC%E8%9F%B2-%E5%BE%9Edcard%E7%B6%B2%E7%AB%99%E7%9C%8B%E7%88%AC%E8%9F%B2%E5%85%A5%E9%96%80-iii-ded52759d922)


# 嘗試抓取一篇Dcard的文章
我們先隨機挑選一篇Dcard上的文章作為練習，我挑選的是[一個人的遊樂園 #麗寶](https://www.dcard.tw/f/funny/p/231029285)，<br>從網址中可以發現這篇文章的編號是231029285，因此我們把編號放在Dcard提供的API網址後面

In [1]:
# 載入使用的套件
import pandas as pd
import requests
from requests_html import HTML
import re

In [6]:
ID = '231030181'
url = 'https://www.dcard.tw/_api/posts/' + ID
# 透過request套件抓下這個網址的資料
requ = requests.get(url)
# 初步檢視抓到的資料結構
requ.json()

{'id': 231030181,
 'title': '警察閃光get',
 'content': '首先謝謝大家對我媽的愛戴，但是目前警察閃光get 的呼聲比較多，所以睡不著的我趕快來交作業😎\n\n\n好啦……直接切入重點，我是如何get 警察閃光?\n\n首先要有一個警察朋友(誤\n我們是因為共同朋友而認識的，絕對不是車禍臨檢或是攔查喔(但我媽篤定一定是我違規……)\n\n而且剛開始的我蠻排斥另一半是警察，因為沒有一個警察能夠無時無刻待在家，而另一半常常要一肩扛起家裡的任何瑣事(因為我爸的朋友是警察，所以從小看到蠻有感觸的，就暗暗發誓以後不想交警察男友)\n\n這就是這口無遮攔的這張嘴😱😱😱\n\n剛開始認識他，我就覺得他是一個蠻暖的男生，因為他分發的區域跟我同個縣市，所以我們又恢復以往的聯繫，漸漸地開始隨便亂聊到可以談心，直到有一天……\n\n他突然傳line跟我說了一句:要一起約去夜市打手槍嗎？\n\n要一起約去夜市打手槍嗎？\n要一起約去夜市打手槍嗎？\n要一起約去夜市打手槍嗎？\n\n喔乾瞬間對這男的完全沒好感……\n但是過了漫長的一段時間，我陰錯陽差的點到跟他的聊天視窗，你們猜猜發生了什麼事?\n\n原來是我眼殘，差點斷送了這個良緣\n\n人家是打:要一起約去夜市打槍嗎?\n人家是打:要一起約去夜市打槍嗎?\n人家是打:要一起約去夜市打槍嗎?\n（因為我超喜歡打生存遊戲，所以原本找我去小試身手😍😍）\n\n是我自己思想太邪惡了嗎？嗚嗚嗚我頓時覺得羞愧立馬回覆他，還騙他說因為在畫設計圖所以一直沒空回覆，誰敢大聲承認自己的邪惡思想啦😱\n\n其實經過一陣子的相處，從他對感情的看法和對家人的態度，還有從我們共同好友聽到他以前在警專時期打電話給家人的樣子，就覺得他是個孝順的孩子（他的電話是一通一通打的，第一通打給爺爺，第二通打給奶奶，再來是爸爸媽媽）\n可能一般人會覺得只要打一通就好啊，幹嘛那麼麻煩? 但是他知道等電話的那個人一定不好受，所以才堅持一通一通打。\n\n後來的我破例了一次，我選擇了他。\n就算他不能無時無刻保護我，不能無時在我身邊陪我，我也要將我自己照顧好不讓他擔心❤\n\n雖然我是個不會甜言蜜語的人，不會一直把愛你到永遠、我們要長長久久之類的話掛嘴邊，因為我比較喜歡用身體力行(沒辦法……我超級實際哈哈)\n\n好了後來的結局有get警察閃光

透過比對網站顯示的內容與上面的輸出後，我們可以從中發現id為文章的編號, title是標題, conten則是內容，其他欄位的說明如下表：

欄位|說明|備註
:-|:-:|:--
ID|編號| 
title|標題| 
content|內容| 
excerpt|摘要| 
createdAt|發布時間| 
updatedAt|更新時間| 
commentCount|留言數| 
forumName|分類|中文
forumAlias|分類|英文
gender|性別| 
likeCount|心情數量| 
reactions|心情細項|把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics|標籤| 

在了解資料結構之後，我們就可以建立函數，讓我們輸入網址後就把資料寫成DataFrame
我們嘗試一下把剛剛抓下來的資料轉換為dataFrame

In [7]:
ID = '231030181'
url = url = 'https://www.dcard.tw/_api/posts/' + ID
requ = requests.get(url)
rejs = requ.json()
pd.DataFrame(
    data=
    [{'ID':rejs['id'],
      'title':rejs['title'],
      'content':rejs['content'],
      'excerpt':rejs['excerpt'],
      'createdAt':rejs['createdAt'],
      'updatedAt':rejs['updatedAt'],
      'commentCount':rejs['commentCount'],
      'forumName':rejs['forumName'],
      'forumAlias':rejs['forumAlias'],
      'gender':rejs['gender'],
      'likeCount':rejs['likeCount'],
      'reactions':rejs['reactions'],
      'topics':rejs['topics']}],
    columns=['ID','title','content','excerpt','createdAt','updatedAt','commentCount',
             'forumName','forumAlias','gender','likeCount','reactions','topics'])

,ID,title,content,excerpt,createdAt,updatedAt,commentCount,forumName,forumAlias,gender,likeCount,reactions,topics
0,231030181,警察閃光get,首先謝謝大家對我媽的愛戴，但是目前警察閃光get 的呼聲比較多，所以睡不著的我趕快來交作業😎...,首先謝謝大家對我媽的愛戴，但是目前警察閃光get 的呼聲比較多，所以睡不著的我趕快來交作業，...,2019-04-05T17:28:38.734Z,2019-04-05T18:29:11.206Z,6,有趣,funny,F,25,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[警察閃光get, 感情]"


在這裡轉寫一個簡單的Crawl函數，讓我們只要輸入文章的ID後，就可以回傳爬下來的文章內容

In [8]:
def Crawl(ID):
    link = 'https://www.dcard.tw/_api/posts/' + str(ID)
    requ = requests.get(link)
    rejs = requ.json()
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt':rejs['updatedAt'],
          'commentCount':rejs['commentCount'],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'likeCount':rejs['likeCount'],
          'reactions':rejs['reactions'],
          'topics':rejs['topics']}],
        columns=['ID','title','content','excerpt','createdAt','updatedAt','commentCount',
                 'forumName','forumAlias','gender','likeCount','reactions','topics']))

嘗試一下透過Crawl函數抓取這篇文章

In [9]:
# 嘗試一下定義的函數是不是能正常使用
Crawl(231030181)

,ID,title,content,excerpt,createdAt,updatedAt,commentCount,forumName,forumAlias,gender,likeCount,reactions,topics
0,231030181,警察閃光get,首先謝謝大家對我媽的愛戴，但是目前警察閃光get 的呼聲比較多，所以睡不著的我趕快來交作業😎...,首先謝謝大家對我媽的愛戴，但是目前警察閃光get 的呼聲比較多，所以睡不著的我趕快來交作業，...,2019-04-05T17:28:38.734Z,2019-04-05T18:29:11.206Z,6,有趣,funny,F,25,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[警察閃光get, 感情]"


Good!函數能正常執行!

# 爬取最熱門的100篇文章
在這邊使用dcard提供便利的API，讓我們可以直接快速爬取資料
[dcard API](https://www.dcard.tw/_api/posts?popular=true&limit=100)
以下簡單說明這個網址
- popular參數：若設定為true，表示按照熱門程度排序，若設定為false，則按照發布時間排序
- limit參數：限定在0-100的數值，表示要抓多少文章
https://www.dcard.tw/_api/posts?popular=true&limit=100

In [29]:
url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Crawl(rejs[i]['id']),ignore_index=True)
print(df.shape)
df

(100, 13)


,ID,title,content,excerpt,createdAt,updatedAt,commentCount,forumName,forumAlias,gender,likeCount,reactions,topics
0,231026207,#幫高調!! 台灣已經進入準戰爭狀態了(持續更新),這不是危言聳聽，你現在所擁有的一切可能都會因此消失，中國不會等你，他已經開始全面入侵。\n\...,這不是危言聳聽，你現在所擁有的一切可能都會因此消失，中國不會等你，他已經開始全面入侵。更新：...,2019-04-05T06:27:33.478Z,2019-04-06T05:17:37.299Z,4143,心情,mood,M,43722,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[中國, 台灣, 戰爭, 資訊戰]"
1,231026876,《大學生了沒》的前班底居然在做這種事！？,🔥 🔥 🔥 摘要 🔥 🔥 🔥 \n前大學生了沒班底Ruby在協助中國政府培植小模、網紅😱😱\...,摘要，前大學生了沒班底Ruby在協助中國政府培植小模、網紅，其實在不危害台灣人的情況下，Ru...,2019-04-05T08:56:58.283Z,2019-04-06T09:45:22.168Z,322,追星,entertainer,M,12358,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[網紅]
2,231026230,男朋友的乾妹被強暴,我男友有一個結拜多年的，也算青梅竹馬吧的女生，他都叫他妹妹～\n\n我們在一起時，我接受了這...,我男友有一個結拜多年的，也算青梅竹馬吧的女生，他都叫他妹妹～我們在一起時，我接受了這件事，我...,2019-04-05T06:31:19.245Z,2019-04-05T06:31:19.245Z,377,感情,relationship,F,9931,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[抒發心情]
3,231025635,我用潤餅畫了一張圖,（看到標題怕大家誤會，不是用潤餅的料畫圖啊，是買了潤餅然後來畫圖\n然後因為這次的主題很有趣...,（看到標題怕大家誤會，不是用潤餅的料畫圖啊，是買了潤餅然後來畫圖，然後因為這次的主題很有趣，...,2019-04-05T04:00:31.216Z,2019-04-05T22:07:20.937Z,108,有趣,funny,F,8791,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[插畫, 美食, 有趣, 小吃]"
4,231028894,What's in my room 3坪雅房改造(圖多慎入),----------------------------4/6 14:16更新花費明細在結尾...,----------------------------4/6 14：16更新花費明細在結尾...,2019-04-05T14:36:40.609Z,2019-04-06T06:17:05.657Z,299,居家生活,house,M,6704,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[WhatsInMyRoom, 居家佈置, 改造, 租屋, 房間]"
5,231026849,我還活著真是奇蹟,更新：\n標題是不是要改成大家還活著真是奇蹟\n下面留言有點可怕\n然後不要不生小孩啦 小孩...,更新：標題是不是要改成大家還活著真是奇蹟，下面留言有點可怕，然後不要不生小孩啦 小孩很可愛，...,2019-04-05T08:50:55.934Z,2019-04-06T05:59:20.957Z,338,有趣,funny,F,6613,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[愛惜自己, 東西要收好, 哪裡有在賣雞蛋冰]"
6,231028072,妳不要一直消費我啦(´･ω･`),國中的時候班上有一個女生 很瘦\n而我又是肉肉的那種（165/57）\n\n*編輯文章一下：...,國中的時候班上有一個女生 很瘦，而我又是肉肉的那種（165/57），*編輯文章一下：大家都覺...,2019-04-05T12:34:59.666Z,2019-04-05T17:44:36.547Z,71,女孩,girl,F,6431,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[]
7,231026606,離開寵物展認養區，哭了,昨天一早跟老公去掃墓、辦事情\n事情辦完後已經4：30了，立馬殺去寵物展\n買6隻米克斯喵寶...,昨天一早跟老公去掃墓、辦事情，事情辦完後已經4：30了，立馬殺去寵物展，買6隻米克斯喵寶的飼...,2019-04-05T07:59:09.789Z,2019-04-05T07:59:09.789Z,212,寵物,pet,F,6342,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[米克斯, 領養代替購買]"
8,231025552,（更）剛交往發現女友在西斯po過文，我提分手。,留言我都有看。\n\n我很喜歡她沒錯，但她抒發情緒的方法錯了，如果因為吵架，就去找其他交友軟...,留言我都有看。我很喜歡她沒錯，但她抒發情緒的方法錯了，如果因為吵架，就去找其他交友軟體的異性...,2019-04-05T03:38:12.800Z,2019-04-06T07:06:12.410Z,379,感情,relationship,M,3509,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[愛情, 分手, 西斯]"
9,231025506,#推薦 尺度開的少女漫,大家也知道一般少女漫畫男女主進度\n頂多只有到親親抱抱\n可能劇情因素會推一兩個畫面是上床\...,大家也知道一般少女漫畫男女主進度，頂多只有到親親抱抱，可能劇情因素會推一兩個畫面是上床，在此...,2019-04-05T03:24:30.953Z,2019-04-05T11:40:28.568Z,94,動漫,acg,F,3411,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[少女漫畫]


#  爬取超過100篇文章
由於API限制最多載入100篇文章，如果我們想要爬更多資料，可以透過before函數與迴圈進行!<br>
before參數後面是接文章的ID，讓我們可以抓取某篇文章之前的資料
而透過迴圈，我們只需要把之前抓到最後一篇文章的ID放入before參數中，我們就可以抓到這篇文章之前100天的文章。

In [30]:
for j in range(10):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Crawl(rejs[i]['id']), ignore_index=True)
print(df.shape)
df

(1100, 13)


,ID,title,content,excerpt,createdAt,updatedAt,commentCount,forumName,forumAlias,gender,likeCount,reactions,topics
0,231026207,#幫高調!! 台灣已經進入準戰爭狀態了(持續更新),這不是危言聳聽，你現在所擁有的一切可能都會因此消失，中國不會等你，他已經開始全面入侵。\n\...,這不是危言聳聽，你現在所擁有的一切可能都會因此消失，中國不會等你，他已經開始全面入侵。更新：...,2019-04-05T06:27:33.478Z,2019-04-06T05:17:37.299Z,4143,心情,mood,M,43722,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[中國, 台灣, 戰爭, 資訊戰]"
1,231026876,《大學生了沒》的前班底居然在做這種事！？,🔥 🔥 🔥 摘要 🔥 🔥 🔥 \n前大學生了沒班底Ruby在協助中國政府培植小模、網紅😱😱\...,摘要，前大學生了沒班底Ruby在協助中國政府培植小模、網紅，其實在不危害台灣人的情況下，Ru...,2019-04-05T08:56:58.283Z,2019-04-06T09:45:22.168Z,322,追星,entertainer,M,12358,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[網紅]
2,231026230,男朋友的乾妹被強暴,我男友有一個結拜多年的，也算青梅竹馬吧的女生，他都叫他妹妹～\n\n我們在一起時，我接受了這...,我男友有一個結拜多年的，也算青梅竹馬吧的女生，他都叫他妹妹～我們在一起時，我接受了這件事，我...,2019-04-05T06:31:19.245Z,2019-04-05T06:31:19.245Z,377,感情,relationship,F,9931,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[抒發心情]
3,231025635,我用潤餅畫了一張圖,（看到標題怕大家誤會，不是用潤餅的料畫圖啊，是買了潤餅然後來畫圖\n然後因為這次的主題很有趣...,（看到標題怕大家誤會，不是用潤餅的料畫圖啊，是買了潤餅然後來畫圖，然後因為這次的主題很有趣，...,2019-04-05T04:00:31.216Z,2019-04-05T22:07:20.937Z,108,有趣,funny,F,8791,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[插畫, 美食, 有趣, 小吃]"
4,231028894,What's in my room 3坪雅房改造(圖多慎入),----------------------------4/6 14:16更新花費明細在結尾...,----------------------------4/6 14：16更新花費明細在結尾...,2019-04-05T14:36:40.609Z,2019-04-06T06:17:05.657Z,299,居家生活,house,M,6704,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[WhatsInMyRoom, 居家佈置, 改造, 租屋, 房間]"
5,231026849,我還活著真是奇蹟,更新：\n標題是不是要改成大家還活著真是奇蹟\n下面留言有點可怕\n然後不要不生小孩啦 小孩...,更新：標題是不是要改成大家還活著真是奇蹟，下面留言有點可怕，然後不要不生小孩啦 小孩很可愛，...,2019-04-05T08:50:55.934Z,2019-04-06T05:59:20.957Z,338,有趣,funny,F,6613,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[愛惜自己, 東西要收好, 哪裡有在賣雞蛋冰]"
6,231028072,妳不要一直消費我啦(´･ω･`),國中的時候班上有一個女生 很瘦\n而我又是肉肉的那種（165/57）\n\n*編輯文章一下：...,國中的時候班上有一個女生 很瘦，而我又是肉肉的那種（165/57），*編輯文章一下：大家都覺...,2019-04-05T12:34:59.666Z,2019-04-05T17:44:36.547Z,71,女孩,girl,F,6431,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[]
7,231026606,離開寵物展認養區，哭了,昨天一早跟老公去掃墓、辦事情\n事情辦完後已經4：30了，立馬殺去寵物展\n買6隻米克斯喵寶...,昨天一早跟老公去掃墓、辦事情，事情辦完後已經4：30了，立馬殺去寵物展，買6隻米克斯喵寶的飼...,2019-04-05T07:59:09.789Z,2019-04-05T07:59:09.789Z,212,寵物,pet,F,6342,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[米克斯, 領養代替購買]"
8,231025552,（更）剛交往發現女友在西斯po過文，我提分手。,留言我都有看。\n\n我很喜歡她沒錯，但她抒發情緒的方法錯了，如果因為吵架，就去找其他交友軟...,留言我都有看。我很喜歡她沒錯，但她抒發情緒的方法錯了，如果因為吵架，就去找其他交友軟體的異性...,2019-04-05T03:38:12.800Z,2019-04-06T07:06:12.410Z,379,感情,relationship,M,3509,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,"[愛情, 分手, 西斯]"
9,231025506,#推薦 尺度開的少女漫,大家也知道一般少女漫畫男女主進度\n頂多只有到親親抱抱\n可能劇情因素會推一兩個畫面是上床\...,大家也知道一般少女漫畫男女主進度，頂多只有到親親抱抱，可能劇情因素會推一兩個畫面是上床，在此...,2019-04-05T03:24:30.953Z,2019-04-05T11:40:28.568Z,94,動漫,acg,F,3411,[{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03'...,[少女漫畫]


In [31]:
# 將資料存到桌面
df.to_excel('C:/Users/TLYu0419/Desktop/Dcard.xlsx')

C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'https://i.imgur.com/Ho4FQpe.jpg
石虎為瀕臨絕種保育類動物，
但苗縣議員卻認為石虎「超過上萬隻了吧」，才會跑到路上被撞死。

苗栗縣議員韓茂賢，
日前在一場苗栗縣政府舉辦的座談會表示，10年前調查石虎有500隻，
若每年以1.5倍的成長來推算，10年之後、
現在「應該會超過上萬隻了吧」，
其結果是
「石虎會跑到路上…路上呢被車子撞」。



韓茂賢認為
「上萬隻的結果是什麼？石虎老是會跑到路上被車子撞」

他表示，為了防止路殺事件，
苗栗縣政府在資源匱乏情況下，
所有的重大建設又被保育團體要求要做安全措施，以防石虎命喪車輪

「可是石虎卻太多了才跑到路上來，怎麼辦？難道可以叫各位不要開車經過道路嗎？」



https://i.imgur.com/Ygi8iCn.jpg

完整版本
https://m.facebook.com/story.php?story_fbid=557119408131749&id=481213905297826&_rdr
https://www.ettoday.net/news/20190404/1415432.htm


後續：
此說法一出引來眾多網友怒批，
今天下午韓茂賢對媒體澄清，
保育團體一直以10年前數字，
至今沒有重新估算族群數量，
未免太怠惰了。

韓茂賢認為，應重新計算族群，
因簡單來算有1.9萬多隻，
但扣掉病死等狀況，推算也有1萬隻。

台灣動物保護執行長何宗勳表示，%20
台灣黑熊幾年前估算只有幾百隻，
但到現在也沒增加，
反問該議員是如何推估石虎成長數量？
有科學證據或相關報告？

何宗勳說，
身為一個民意代表有監督地方政府權利，
尤其在愛護保護動物風潮高漲世代，
更應該是先好好了解情況再來討論。


😡

苗栗此議員的下限.....我看不到' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\TL_Yu\Ana

C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'https://www.dcard.tw/f/travel/p/230900611-%EF%BC%83%E8%87%AA%E5%8A%A9%EF%BC%83%E9%81%8A%E8%A8%98-%E9%9F%93%E5%9C%8B%E4%BA%94%E5%A4%A9%E5%9B%9B%E5%A4%9C%E9%81%8A 

對遊記部分有興趣的可以前往上一篇文章⬆

*

依舊先來個前情提要，我是個女孩🙋 

先放上所有戰利品~~


http://i.imgur.com/lUlBcKA.jpg

  Innisfree


http://i.imgur.com/UtEW6SQ.jpg


本來只是想買個修容、打亮而已
結果為了湊免稅又多買了好多哈哈😂😂😂

詳細部分⬇


http://i.imgur.com/4xYshHM.jpg

修容、打亮的部分都是偏自然妝感
喜歡自然妝感的水水們可以買!!!
打亮看起來是粉金色不過實際上色在手上偏白金~


http://i.imgur.com/SjylTOx.jpg

順便買了打亮跟修容刷
想當然不用多說~大家都知道Innisfree家的刷子是出了名的柔軟、不刺臉
說有多好用就有多好用d(`･∀･)b
修容刷我覺得很蓬鬆柔軟我應該也會拿來當蜜粉刷使用



http://i.imgur.com/dJKCE4a.jpg

湊免稅時也被櫃姐推銷了香氛

木棉花味

很清新自然的香味
我以為裡面有附擴香棒結果沒買இдஇ


最後要結帳時店員說好像湊滿多少就多送綠茶組合
順便加購了旁邊的笑臉手帳
其實我原本以為裡面是笑臉刷具組結果回家開箱才發現是手帳QQ

全額91000韓幣
扣稅後71000韓幣
換算台幣 1930元左右

  Holika Holika


http://i.imgur.com/3o14dR2.jpg

本來想買他們新出的眼影盤01號
結果每個地方都跑遍了全部都 sold out 超傷心(´;ω;`)


http://i.imgur.com/WTMX3lQ.jpg

大盤的還沒試色過，顏色都好美捨不得搓QQ(' w

C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'https://youtu.be/5KtCsmwYDXY%20

不知道大家有覺得張書豪
演愛情戲都是飾演悲情另一半的角色嗎XD

比悲傷電影裡飾演工具人牙醫
看如果愛重來的預告感覺也很虐，
都是成全別人的那個人哈哈

http://i.imgur.com/NOgsGZK.jpg

%20https://www.facebook.com/iottdejavu/videos/402903337194512/%20

最新預告好像是跟柯佳嬿的婚姻生活片段
張書豪真的很會演，完全有被最後萌到，感覺是新好男人阿~

http://i.imgur.com/jJYQ5xs.jpg

打算追完%20' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'https://www.dcard.tw/f/boyslove/p/231021921?ref=android


03

　　羅研大約在晚上十點左右收好行李，他東西不多，就收了幾件換洗衣物、樂譜、筆記本，還有一把吉他。

　　洗好澡的時候已經十點半了，他抬頭看了看掛鐘，有些失望地想溫敬暉今晚大概是不會來了。

　　他們維持這種關係三年，最久一次沒見面也僅僅半個月，那時羅研已經跟了溫敬暉一段時間，原先那點戰戰兢兢被磨去了不少。溫敬暉到歐洲出差，回國後順路繞到羅研這，本意是想把隨手買的禮物先拿給他，順道休息一下調調時差，羅研卻一點也不在乎那些禮物，他粘黏膩膩地纏上溫敬暉，撩起火後主動獻身。

　　那時候也才不過半個月而已，他就已經想人想得不行了，這次要一個月啊……

　　羅研還在苦惱要怎麼度過見不著溫敬暉的一個月，大門處忽然傳來細碎的開鎖聲，他頓了一下，三兩步跑出房門，和正推開大門的

C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'https://youtu.be/pdDE5Z5Nz-A

這就是我之前在其他人留言版說的，接下來是銀行債務問題！

早在幾年前P2P倒閉潮，再來到近期O2O，倒閉潮，還有房產泡沫，超印鈔票，一帶一路政策失利還有潛在的銀行超支借貸，債務，等問題，將會繼續連環爆，從去年500萬間公司倒閉，還有政府提高稅收，就知道了。

總之中國經濟神話將會繼續崩，讓我們繼續看下去！

另外美中貿易戰我不信川普不會持續進行就是了🤔

某黨還在把賭注押在中國身上😑

真的還是建議大家到東南亞卡位，想賺大錢那邊機會將會越來越多！

而且還能夠國際化😊' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'https://i.imgur.com/9m5MfFD.jpg
嗨嗨大家
來分享最近被燒到的隱眼
愛能視應該比較少人戴過
我自己很喜歡他們家的森林系列
有鳥跟松鼠的那個包裝，不僅包裝好看
出的兩色我都超愛的
尤其是金茶棕超混血，有人敲我再來拍～

因為我眼珠比較大顆，所以我不喜歡戴黑邊的
平常都被誤認有戴放大

這次戴的是他們新出的系列%20偷心
是月拋，這樣以後戴就不會那麼燒錢了
只是說他們這個包裝我實在是⋯⋯
https://i.imgur.com/bcrdYUq.jpg
https://i.imgur.com/nSuqLPw.jpg

我先戴棕色的，灰色我還沒想好要怎麼戴
也不知道戴起來怎樣，過陣子再來試
https://i.imgur.com/FSlhy0M.jpg
https://i.imgur.com/24k0LID.jpg

這個棕我真的覺得很可以
他看起來好像很大，但我為什麼說他是小直徑
因為我的眼珠有點遮不住，微微跑出黑色

C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'https://i.imgur.com/mhsZ83b.jpg
【復仇者聯盟4：終局之戰】與前作【無限之戰】一樣都全程使用IMAX攝影機來拍攝，從拍攝到播放都使用同一個系統絕對會是最佳的觀影體驗。

但在最近確定了台灣地區的IMAX版「僅有『3D』版本」，這讓許多影迷、粉絲崩潰。
https://i.imgur.com/b3beDMU.jpg

後續美麗華也再次證實這個消息
https://i.imgur.com/qQ3llrR.jpg
https://i.imgur.com/u6n2qqt.jpg

之前【無限之戰】的時候還有2D可以選擇來觀賞，為什麼這次僅提供3D？我有個推測

第一、擁有IMAX的影城都只有一廳。
第二、片長182min，已達到過長，一天排4～5場次。(能5場播完都快凌晨4～5點)
第三、檔期過短，下一部要上IMAX的是【偵探皮卡丘】(5/9)，【終局之戰】可以獨佔IMAX戲院約莫僅15天。

而【終局之戰】是一部挑戰影史票房前3名的電影，我想為了鞏固海外的票房，所以讓IMAX僅有3D版本。

哎呀，看3小時的3D眼睛應該是會被榨乾吧……還有可能會把3D眼鏡哭得整個都是。我終於知道第二支預告不斷重複「不計一切代價」的梗了，原來米老鼠早就想好了。

上映倒數19天，4/12中午12點各大戲院開始預售。' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'https://i.imgur.com/PcJWZk6.jpg

參觀古城不一定要具備豐富的歷史記憶，但一定要有充分的想像力！

以弗所一開始為希臘移民所居，當時的人民崇尚阿提米斯，由神殿的精緻雕刻可以看出當時在藝術及建築已有極高的文明。後來的亞

C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'http://i.imgur.com/3hTR5EN.jpg

《他們不再老去》是一部戰爭歷史紀錄片，由彼得·傑克森執導，劇情講述第一次世界大戰歐洲戰場的西方戰線，英國軍人對戰德軍的過程，利用當時留下的許多影像與圖片，配合退伍軍人的訪談內容，為這段過去歷史做了最真實的紀錄，把那些我們平常不無法接觸的戰地真相，清楚地攤在世人眼前。

導演彼得·傑克森經過《魔戒》、《哈比人》這種充滿奇幻元素的電影之後，誰能料到他竟會回頭著手製作一部如此寫實，甚至全部素材都是取自收錄在英國帝國戰爭博物館內檔案的紀錄片？雖說《他們不再老去》所描寫的一戰已經過了一百多年，不過在我看完走出戲院之後，馬上明白這部電影的最大價值，絕對不在於他能帶給我們多少視覺震撼和娛樂效果，而是觀眾是否能通過片中的人物、事件或劇情，從中多少對當時情況有一點了解，甚至是影響我們對事物看法的深刻體悟或啟發。

http://i.imgur.com/0XtycAp.jpg

我必須老實說，《他們不再老去》作為一部電影是非常無聊的，除了故事幾乎沒有任何的高低起伏，連電影本身的畫面和聲音也相當平淡，但即便如此，它所帶給我的衝擊卻是如此的強烈，完全無法抵抗地讓我熱淚盈眶。其中一方面是他們的遭遇太令人難過，而另一方面則是對導演與其他劇組人員的深深欽佩，就我而言，一部紀錄片要成功的最大因素，就是導演如何運用手中取得的素材，把它們歸納整理並串成一個完整、循序漸進的故事。

因此光是劇組在全部超過2200小時的影片，還有600小時訪談檔案的精挑細選相信就必須耗費大量心血，更不用說《他們不再老去》的主打內容，就是把這些瑣碎、黑白、無聲且模糊的原始素材，透過曠日廢時的影像修復、上色，再由嘴型還原士兵當時的談話內容，並以擬音技術模擬出包括人聲、行軍、槍砲、環境音等等我們在片中聽見的所有音效，雖然並非毫無破綻，但依舊能感受到《他們不再老去》技術層面的強大之處。

http://i.imgur.com/zbulUuj.jpg

不過在如此高水準的成果之下，又不禁讓我思考這些豐富的資料到底有什麼價值，能讓劇組付出這麼多心血和努力，況且在看到片中這麼多

C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'https://www.cna.com.tw/news/afe/201904050030.aspx
看時事版10有8、9都是在講中共，既然那麼討厭中共還那麼關心幹嘛？難道是工讀生？
趕快回來關心我國製造業吧，如果真的愛台灣就關心一下各行各業發展，看看貿易戰中我方能否得利，關心人家中共美國幹嘛？他們得利也不會分我們啦
—————————————————-

更：看到你們那麼多人記得我，我以後還是多多發文好了，讓你們映像深刻😬
放心！我一向對我的言論很有信心，聽說連暴政公社都有我啦？幫我多宣傳👍
記得增加自己的媒體釋讀能力喔～' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\TL_Yu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:910: UserWarning: Ignoring URL 'https://i.imgur.com/ZRdqcs7.jpg

%20看了某網友po這圖獲得不少好評!

穿裙總統，辣台妹?

這新聞超無言，中共飛來打個嘴炮發個廢文就叫硬?%20
國內經濟軟趴趴，
房地產租金，生育率，免服役靠徵兵，國防是用嘴守護的?

2020前嘴炮打好打滿，弄一堆表面功夫，台灣不實質提升每天打嘴炮，中共就怕你?

怎不回擊，下次再飛來直接擊落

中共戰鬥機敢飛美國?川普會像蔡英文一樣臉書po個空談文，毫無作為?

人家上台就是決心跟中共PK經濟

中美貿易戰快把對岸打趴了

台灣不到那大國趴數，但最少小國可以努力執政讓人民有感

讓人民安樂，拼經濟，房價有感大幅砍，幸福指數提高才叫硬起來!

你敢和像砍軍公教年金，把台灣高房價砍到地板價，我就認同你

不可能一因為政客們通通也買在台北，哪個會真心當政治家?%20

把風電非核好幾兆預算用在全國學子能免費從幼稚園唸完大學還有找，用心拼生育率政策

別浪費稅金在那建一堆
上兆

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/TLYu0419/Desktop/Dcard.xlsx'